In [26]:
import torch
import sys
sys.path.insert(0, '/home/akiva/FCNX-Ensembling')

from FCN3Network import FCN3NetworkEnsembleErf

def custom_mse_loss(outputs, targets):
    diff = outputs - targets
    return torch.sum(diff * diff)
    # Initialize FCN3NetworkEnsembleErf with ens=10, d=20, P=20, N=400
  

In [ ]:
device = 'cuda:1'
ens = 100
d = 20
P = 10000
N = 10
model = FCN3NetworkEnsembleErf(d, N,N,P, ens=ens, weight_initialization_variance=(1/d,1/N, 1/(N)), device='cuda:1')
torch.manual_seed(613)
# Create data on GPU directly and pin memory
X = torch.randn((P, d), device=device)
Y = X[:, 0].unsqueeze(-1)






NameError: name 'FCN3NetworkEnsembleErf' is not defined

In [43]:
Yr = Y.unsqueeze(-1)
Yr.shape


torch.Size([10000, 1, 1])

In [44]:
outputs = model(X)
loss = custom_mse_loss(outputs, Yr)
avg_loss = loss / (ens * P)
avg_loss

tensor(1.3074, device='cuda:1', grad_fn=<DivBackward0>)

In [105]:
P

20

In [103]:
((outputs-Y.unsqueeze(-1))**2).sum() / (10 * P * 2) 

tensor(0.4380, device='cuda:1', grad_fn=<DivBackward0>)

In [85]:
((outputs-Y.unsqueeze(-2))**2).sum()

tensor(175.1940, device='cuda:1', grad_fn=<SumBackward0>)

In [101]:
(outputs-Y).shape

torch.Size([20, 20, 10])

In [100]:
((outputs-Y)**2).sum()/ (10 * P * 2) 

tensor(7.8752, device='cuda:1', grad_fn=<DivBackward0>)

In [94]:
ens

100

In [97]:
(model(X)**2).sum() / (P * 2 * 10) 

tensor(0.1222, device='cuda:1', grad_fn=<DivBackward0>)

In [88]:
torch.std(X)

tensor(1.0196, device='cuda:1')

In [4]:
torch.cuda.is_available()

True

In [8]:
if not torch.cuda.is_available():
    print("CUDA is not available. Using CPU.")

num_gpus = torch.cuda.device_count()
if num_gpus == 0:
    print("No CUDA devices found. Using CPU.")

least_memory_allocated = float('inf')
least_utilized_device = None

print(f"Checking {num_gpus} GPU(s) for utilization...")

for i in range(num_gpus):
    device = torch.device(f"cuda:{i}")
    # Set the current device to query its memory
    torch.cuda.set_device(device)
    
    # Get allocated memory (bytes)
    # memory_allocated() returns the total bytes that PyTorch has allocated on the GPU.
    # memory_reserved() returns the total bytes that PyTorch has reserved (cached) on the GPU.
    # allocated_bytes = torch.cuda.memory_allocated(device)
    # reserved_bytes = torch.cuda.memory_reserved(device)
    
    # For simplicity, we'll use memory_allocated as a primary metric.
    # You might consider memory_reserved or a combination for more nuanced decisions.
    current_allocated_memory = torch.cuda.memory_allocated(device)
    
    print(f"  Device {i} ({torch.cuda.get_device_name(i)}): {current_allocated_memory / (1024**2):.2f} MB allocated")

    if current_allocated_memory < least_memory_allocated:
        least_memory_allocated = current_allocated_memory
        least_utilized_device = device

if least_utilized_device:
    print(f"\nSelected device: {least_utilized_device} (Least allocated memory: {least_memory_allocated / (1024**2):.2f} MB)")
else:
    # Fallback, should not happen if num_gpus > 0
    print("Could not determine least utilized GPU. Falling back to CPU.")
    print('CPU')

Checking 2 GPU(s) for utilization...
  Device 0 (NVIDIA GeForce RTX 4090): 0.00 MB allocated
  Device 1 (NVIDIA GeForce RTX 4080): 0.00 MB allocated

Selected device: cuda:0 (Least allocated memory: 0.00 MB)
